### Import libraries 

In [10]:
from openai import OpenAI
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
client = OpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.proxyapi.ru/openai/v1",
)

chat_completion = client.chat.completions.create(
    model="gpt-4o-mini", messages=[{"role": "user", "content": "Hello world"}]
)

In [1]:
import os
import sys
from langchain.docstore.document import Document

from typing import List
from rank_bm25 import BM25Okapi
import numpy as np


sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..'))) # Add the parent directory to the path sicnce we work with notebooks
from helper_functions import *
from evalute_rag import *

from dotenv import load_dotenv
load_dotenv()
# Load environment variables from a .env file

# Set the OpenAI API key environment variable
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

/Users/kartashoffv/Documents/pet-projects/test/venv/lib/python3.10/site-packages/deepeval/__init__.py:49: UserWarning: You are using deepeval version 1.1.6, however version 1.3.2 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(


In [35]:
import pandas as pd
df = pd.read_excel("/Users/kartashoffv/Documents/pet-projects/test/rutube_helper_train/rag/02_Реальные_кейсы.xlsx")
df['concated_query_question'] = "[" + df['Классификатор 1 уровня'] + ", " + df['Классификатор 2 уровня'] + "]" + "\Запрос: " + df['Вопрос пользователя'] 
df['concated_query_answer'] ="[" + df['Классификатор 1 уровня'] + ", " + df['Классификатор 2 уровня'] + "]" + "\nЗапрос: " + df['Вопрос из БЗ'] + "\nОтвет: " + df['Ответ из БЗ'] 
df.head()

,Вопрос пользователя,Ответ сотрудника,Вопрос из БЗ,Ответ из БЗ,Классификатор 1 уровня,Классификатор 2 уровня,concated_query_question,concated_query_answer
0,Здравствуйте! Можно уточнить причины Правилhtt...,Добрый день!\nЧто нельзя публиковать на RUTUBE...,Что нельзя публиковать на RUTUBE?,Чужой контент без разрешения автора или правоо...,МОДЕРАЦИЯ,Отклонение/блокировка видео,"[МОДЕРАЦИЯ, Отклонение/блокировка видео]\Запро...","[МОДЕРАЦИЯ, Отклонение/блокировка видео]\nЗапр..."
1,"Добрый вечер, какой топ причин блокировки виде...",Добрый вечер!\nЧто заперщено публиковать на RU...,Что нельзя публиковать на RUTUBE?,Чужой контент без разрешения автора или правоо...,МОДЕРАЦИЯ,Отклонение/блокировка видео,"[МОДЕРАЦИЯ, Отклонение/блокировка видео]\Запро...","[МОДЕРАЦИЯ, Отклонение/блокировка видео]\nЗапр..."
2,"Все пишут, что монетизация на рутубе отключает...","Добрый день! \nМонетизация может отключиться, ...",Почему могут отключить монетизацию из-за автор...,"Монетизация может отключиться, если на вашем к...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации,"[МОНЕТИЗАЦИЯ, Отключение/подключение монетизац...","[МОНЕТИЗАЦИЯ, Отключение/подключение монетизац..."
3,Что запрещено в монетизации и что можно выклад...,"Здравствуйте!\nМонетизация может отключиться, ...",Почему могут отключить монетизацию из-за автор...,"Монетизация может отключиться, если на вашем к...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации,"[МОНЕТИЗАЦИЯ, Отключение/подключение монетизац...","[МОНЕТИЗАЦИЯ, Отключение/подключение монетизац..."
4,"Чтобы не отключали монетизацию, надо, чтобы я ...","Для монетизации можно использовать то, что вы ...",Почему могут отключить монетизацию из-за автор...,"Монетизация может отключиться, если на вашем к...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации,"[МОНЕТИЗАЦИЯ, Отключение/подключение монетизац...","[МОНЕТИЗАЦИЯ, Отключение/подключение монетизац..."


In [70]:
from langchain_community.embeddings import HuggingFaceEmbeddings

df['ids'] = [f"id_{i}" for i in range(len(df))]
df['sources'] = [f"link_{i}" for i in range(len(df))]

test_data = {
    "docs": df["concated_query_answer"].tolist(),
    "sources": df["sources"].tolist(),
    "ids": df["ids"].tolist()
}
# embeddings = OpenAIEmbeddings(
#         model="text-embedding-3-large",
#         api_key=os.getenv("OPENAI_API_KEY"),
#         openai_api_base="https://api.proxyapi.ru/openai/v1"
#     )
embeddings = HuggingFaceEmbeddings(model_name = "deepvk/USER-bge-m3")


def create_vectorstore_with_metadata(test_data):
    documents = []
    for doc, source, doc_id in zip(test_data["docs"], test_data["sources"], test_data["ids"]):
        documents.append(Document(page_content=doc, metadata={"source": source, "id": doc_id}))
    
    vectorstore = FAISS.from_documents(documents, embeddings)

    return vectorstore, documents


### Create vectorstore and get the chunked documents

In [38]:
vectorstore, cleaned_texts = create_vectorstore_with_metadata(test_data)

### Create a bm25 index for retrieving documents by keywords

In [42]:
def create_bm25_index(documents: List[Document]) -> BM25Okapi:
    """
    Create a BM25 index from the given documents.

    BM25 (Best Matching 25) is a ranking function used in information retrieval.
    It's based on the probabilistic retrieval framework and is an improvement over TF-IDF.

    Args:
    documents (List[Document]): List of documents to index.

    Returns:
    BM25Okapi: An index that can be used for BM25 scoring.
    """
    # Tokenize each document by splitting on whitespace
    # This is a simple approach and could be improved with more sophisticated tokenization
    tokenized_docs = [doc.page_content.split() for doc in documents]
    return BM25Okapi(tokenized_docs)

In [43]:
bm25 = create_bm25_index(cleaned_texts) # Create BM25 index from the cleaned texts (chunks)

### Define a function that retrieves both semantically and by keyword, normalizes the scores and gets the top k documents

In [67]:
# def fusion_retrieval(vectorstore, bm25, query: str, k: int = 5, alpha: float = 0.5) -> List[Document]:
#     """
#     Perform fusion retrieval combining keyword-based (BM25) and vector-based search.

#     Args:
#     vectorstore (VectorStore): The vectorstore containing the documents.
#     bm25 (BM25Okapi): Pre-computed BM25 index.
#     query (str): The query string.
#     k (int): The number of documents to retrieve.
#     alpha (float): The weight for vector search scores (1-alpha will be the weight for BM25 scores).

#     Returns:
#     List[Document]: The top k documents based on the combined scores.
#     """
#     # Step 1: Get all documents from the vectorstore
#     all_docs = vectorstore.similarity_search("", k=vectorstore.index.ntotal)

#     # Step 2: Perform BM25 search
#     bm25_scores = bm25.get_scores(query.split())

#     # Step 3: Perform vector search
#     vector_results = vectorstore.similarity_search_with_score(query, k=len(all_docs))
    
#     # Step 4: Normalize scores
#     vector_scores = np.array([score for _, score in vector_results])
#     vector_scores = 1 - (vector_scores - np.min(vector_scores)) / (np.max(vector_scores) - np.min(vector_scores))

#     bm25_scores = (bm25_scores - np.min(bm25_scores)) / (np.max(bm25_scores) - np.min(bm25_scores))

#     # Step 5: Combine scores
#     combined_scores = alpha * vector_scores + (1 - alpha) * bm25_scores  

#     # Step 6: Rank documents
#     sorted_indices = np.argsort(combined_scores)[::-1]
    
#     # Step 7: Return top k documents
#     return [all_docs[i] for i in sorted_indices[:k]]

def fusion_retrieval(documents: List[Document], vectorstore, bm25, query: str, k: int = 5, alpha: float = 0.5) -> List[Document]:
    """
    Perform fusion retrieval combining keyword-based (BM25) and vector-based search.

    Args:
    documents (List[Document]): List of all documents in the same order as the BM25 and vectorstore.
    vectorstore (VectorStore): The vectorstore containing the documents.
    bm25 (BM25Okapi): Pre-computed BM25 index.
    query (str): The query string.
    k (int): The number of documents to retrieve.
    alpha (float): The weight for vector search scores (1-alpha will be the weight for BM25 scores).

    Returns:
    List[Document]: The top k documents based on the combined scores.
    """
    # Step 1: Compute BM25 scores for all documents
    bm25_scores = bm25.get_scores(query.split())

    # Step 2: Compute vector scores for all documents
    query_embedding = vectorstore.embedding_function(query)
    
    # Retrieve embeddings for all documents
    document_embeddings = []
    for doc in documents:
        # Assuming each document has an associated embedding stored or can be retrieved
        doc_embedding = vectorstore.embedding_function(doc.page_content)
        document_embeddings.append(doc_embedding)
    
    # Compute cosine similarities between query and all document embeddings
    from sklearn.metrics.pairwise import cosine_similarity
    vector_scores = cosine_similarity([query_embedding], document_embeddings)[0]
    
    # Step 3: Normalize scores
    vector_scores_norm = (vector_scores - np.min(vector_scores)) / (np.max(vector_scores) - np.min(vector_scores) + 1e-8)
    bm25_scores_norm = (bm25_scores - np.min(bm25_scores)) / (np.max(bm25_scores) - np.min(bm25_scores) + 1e-8)

    # Step 4: Combine scores
    combined_scores = alpha * vector_scores_norm + (1 - alpha) * bm25_scores_norm

    # Step 5: Rank documents
    sorted_indices = np.argsort(combined_scores)[::-1]

    # Step 6: Return top k documents
    return [documents[i] for i in sorted_indices[:k]]


In [69]:
# Assuming 'documents' is your list of Document objects
bm25 = create_bm25_index(cleaned_texts)

# Assuming 'vectorstore' is your vectorstore with an 'embedding_function' attribute
top_docs = fusion_retrieval(cleaned_texts, vectorstore, bm25, query, k=5, alpha=0.5)
top_docs
# Now, 'top_docs' should contain the documents that best match your query,
# including those with classification labels like (МОДЕРАЦИЯ, Отклонение/блокировка видео)


TypeError: 'HuggingFaceEmbeddings' object is not callable

### Use Case example

In [46]:
concated_query = df.iloc[0]['concated_query_question']
basic_q = df.iloc[0]['Вопрос пользователя']

In [12]:
q = df.iloc[0].question
print(q)

Что нельзя публиковать на RUTUBE?


In [57]:
print("TRUE SRC IS: ", df.iloc[0]['sources'], "\n********")
basic_q = df.iloc[0]['Вопрос пользователя']

# Query
query = basic_q

# Perform fusion retrieval
top_docs = fusion_retrieval(vectorstore, bm25, query, k=5, alpha=0.5)
docs_source = [doc.metadata['source'] for doc in top_docs]
show_context(docs_source)

TRUE SRC IS:  link_0 
********
Context 1:
link_204


Context 2:
link_519


Context 3:
link_391


Context 4:
link_362


Context 5:
link_392




In [66]:
print("TRUE SRC IS: ", df.iloc[0]['sources'], "\n********")

concated_query = df.iloc[0]['concated_query_question']
print(f'concated_query is {concated_query}')

# query = concated_query
query = concated_query

# Perform fusion retrieval
top_docs = fusion_retrieval(vectorstore, bm25, query, k=5, alpha=0.5)
top_docs

TRUE SRC IS:  link_0 
********
concated_query is [МОДЕРАЦИЯ, Отклонение/блокировка видео]\Запрос: Здравствуйте! Можно уточнить причины Правилhttps://rutube.ru/info/taboo_agreement/ по которым удаляются ролики? что за нарушение правил RUTUBE ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ 


[Document(metadata={'source': 'link_437', 'id': 'id_437'}, page_content='[ВИДЕО, Встраивание видео]\nЗапрос: Какие настройки параметров можно сделать в коде вставки\nОтвет: После вставки кода видео в код страницы в него можно вносить дополнительные изменения параметров, меняя значения в коде вставки.\nПодробнее:\n\nwidth="864" height="486" — ширина и высота окна плеера,\nhttps://rutube.ru/play/embed/7716bd3e665725c3c008ae7ab4ff02e2 — эмбедироваванная ссылка на видео,\nskinColor=e53935 — основной цвет интерфейса (красный),\nt=300 — время в секундах, с которого начнётся воспроизведение (300 секунд = 5 минут),\nstopTime=480 — время в секундах, на котором воспроизведение завершится (480 секунд = 8 минут),\nframeborder — параметр, отвечающий за активацию рамки по периметру окна плеера. Может задаваться значением 1 (вкл) и 0 (выкл, по умолчанию),\nallow="clipboard-write; autoplay" — параметры управления буфером копирования и автоматическим запуском видео, если выполнены все условия для автом

In [63]:
top_docs

[Document(metadata={'source': 'link_423', 'id': 'id_423'}, page_content='[ТРАНСЛЯЦИЯ, Управление трансляцией]\nЗапрос: Как настроить OBS Studio для проведения трансляций на RUTUBE? \nОтвет: Вывод \n1.1 ТРАНСЛЯЦИЯ\n\nУправление битрейтом – CBR\n\nБитрейт – 6000 Kbps\n\nИнтервал ключевых кадров – 1s (это обновление кадров)\n\nПредустановка использования ЦП– Slower\n\nПрофиль – high\n\n1.2 ЗАПИСЬ\n\nКодировщик видео – выбрать в соответствии с имеющейся видеокартой \n\nДля NVIDIA есть соответствующие кодеки:\n\n-NVIDIA NVENC H.264\n\n-NVIDIA NVENC HEVC\n\nДля других видеоадаптеров можно выбрать любой другой\n\nУправление битрейтом – CQP\n\nЗначение CQ – 15\n\nИнтервал ключевых кадров – 2 s\n\nПредустановка – Медленное (хорошее качество)\n\nРегулировка – Высокое качество\n\n1.3 БУФЕР ПОВТОРНА НЕ МЕНЯЕМ\n\n\n\nАудио\n2.1 ГЛОБАЛЬНЫЕ УСТРОЙСТВА АУДИО\n\nМикрофон/дополнительный звук - тут добавляем микрофон, который подключен к ПК, он будет основным микрофоном на трансляции\n\n\n\n2.2 ИЗМЕРИТЕЛ

In [33]:
top_docs = fusion_retrieval(vectorstore, bm25, query, k=5, alpha=0.1)
top_docs

[Document(metadata={'source': 'link_169', 'id': 'id_169'}, page_content='Нет ограничений. Рекомендуются бесплатные ПО OBS Studio (https://obsproject.com/ru) и VMix (https://www.vmix.com/).'),
 Document(metadata={'source': 'link_197', 'id': 'id_197'}, page_content='Сканы паспорта, ИНН, СНИЛС и справка о постановке на учёт в налоговой или справка из приложения «Мой налог», если вы самозанятый/ая. Для ИП и юрлица нужно будет добавить документы о регистрации и выписку из ЕГРИП/ЕГРЮЛ.'),
 Document(metadata={'source': 'link_35', 'id': 'id_35'}, page_content='На данный момент в приложении такой возможности нет, но вы можете подключиться к партнерской программе в веб-версии Студии в разделе "Монетизация": ссылка Подробнее про подключение к партнерской программе: https://rutube.ru/info/monetization'),
 Document(metadata={'source': 'link_147', 'id': 'id_147'}, page_content='Необходимо соблюдать условия Пользовательского соглашения: https://rutube.ru/info/agreement/. Модераторы могут заблокироват

In [17]:
import math
import re


def compute_ndcg(ground_truth_urls: list, retrieved_urls: list, k=None) -> float:
    """
    Функция для вывода NDCG для одного запроса.

    Parameters:
    ground_truth_urls (list): Список URL-адресов (ground truth).
    retrieved_urls (list): Список URL-адресов, извлеченных ретриверов.
    k (int, optional): Позиция ранга, до которой рассчитывается NDCG.

    Returns:
    float: Оценка NDCG для запроса.
    """
    if k is None:
        k = len(retrieved_urls)

    dcg = 0.0
    for i in range(min(k, len(retrieved_urls))):
        rel_i = 1 if retrieved_urls[i] in ground_truth_urls else 0
        if rel_i > 0:
            dcg += rel_i / math.log2(i + 2)

    idcg = 0.0
    for i in range(min(k, len(ground_truth_urls))):
        idcg += 1 / math.log2(i + 2)

    ndcg = dcg / idcg if idcg > 0 else 0.0
    return ndcg


In [24]:
top_docs
sources = [doc.metadata['source'] for doc in top_docs]
sources

['link_169', 'link_197', 'link_35', 'link_63', 'link_286']

In [25]:
new_columns = {'fusion_retrieval_bge': []}

for idx, row in df.iterrows():
    user_query = row['question']
    top_docs = fusion_retrieval(vectorstore, bm25, user_query, k=5, alpha=0.5)
    result = [doc.metadata['source'] for doc in top_docs]
    ids = result
    
    new_columns['fusion_retrieval_bge'].append(ids)    

df['fusion_retrieval_bge'] = new_columns['fusion_retrieval_bge']

In [31]:
df['ndcg_score'] = df.apply(lambda row: compute_ndcg([row['sources']], row['fusion_retrieval_bge']), axis=1)
df['ndcg_score'].mean()

0.008642728114441518